In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import sleep

from pandas_datareader import data as pdr
import pandas_ta as ta #commented out for me for now, I (Ben) had some dependency issues

import yfinance as yf
# import quandl as qd # not used, ignor ples
from backtesting import Backtest, Strategy

Loading BokehJS ...

In [2]:
yf.pdr_override() # for use with pandas-datareader, optional

uncomment below if you need to update/generate your monthly and daily csv files.

In [47]:
# ticker = pd.read_csv('ticker.csv')['Ticker']
# tickers = ticker.to_list() # This is a list of all tickers in the SP500
# tickers = [x.replace('.','-') for x in tickers] # yahoo has '-' instead of '.' for tickers, eg BRK.B

# ## below is how I got monthly and daily pandas dataframes of all stocks in one huge dict.  
# monthly_sp500 = {}
# daily_sp500 = {}
# for tkr in tickers: # run all 500 at your own risk, it takes a while lol
# # for tkr in tickers[:10]:
#     print('Grabbing ' + tkr + " data!")
#     monthly_sp500[tkr] = pdr.get_data_yahoo(tkr, start="2010-01-01", interval = "1mo")
#     daily_sp500[tkr] = pdr.get_data_yahoo(tkr,start="2010-01-01")
#     sleep(.1) # not planning on ddos-ing yahoo today
# monthly_sp500['AAPL'] # take AAPL, for example

# ## everything in one stupid large dataframe
# sp500m = monthly_sp500[tickers[0]] # monthly
# sp500m['Name'] = tickers[0]

# sp500d = daily_sp500[tickers[0]] # daily
# sp500d['Name'] = tickers[0]

# # for tkr in tickers[1:10]:
# for tkr in tickers[1:]:
#     df1 = monthly_sp500[tkr] # monthly
#     df1['Name'] = tkr
#     sp500m = sp500m.append(df1)

#     df2 = daily_sp500[tkr] # daily
#     df2['Name'] = tkr
#     sp500d = sp500d.append(df2)

# sp500m.to_csv('SP500_monthly.csv') # run/uncomment some of these to save these to csv
# sp500d.to_csv('SP500_daily.csv')

**********************]  1 of 1 completed
Grabbing DPZ data!
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Grabbing GPC data!
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Grabbing J data!
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Grabbing ABMD data!
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Grabbing K data!
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Grabbing TDY data!
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Grabbing WAB data!


In [73]:
# df = pd.read_csv('SP500_monthly.csv').infer_objects().dropna()
df = pd.read_csv('SP500_daily.csv').infer_objects().dropna()
# df = sp500m.infer_objects().dropna()#.reset_index() # dataframe of all we basically want, OHLC data w/ adjusted close. didnt set any ind
df

,Date,Open,High,Low,Close,Adj Close,Volume,Name
0,2010-01-04,7.622500,7.660714,7.585000,7.643214,6.583586,493729600.0,AAPL
1,2010-01-05,7.664286,7.699643,7.616071,7.656429,6.594968,601904800.0,AAPL
2,2010-01-06,7.656429,7.686786,7.526786,7.534643,6.490066,552160000.0,AAPL
3,2010-01-07,7.562500,7.571429,7.466071,7.520714,6.478067,477131200.0,AAPL
4,2010-01-08,7.510714,7.571429,7.466429,7.570714,6.521136,447610800.0,AAPL
...,...,...,...,...,...,...,...,...
1365206,2021-03-04,23.020000,23.330000,22.389999,22.770000,22.770000,345900.0,NWS
1365207,2021-03-05,22.940001,23.549999,22.510000,23.500000,23.500000,515500.0,NWS
1365208,2021-03-08,23.500000,23.760000,22.980000,23.230000,23.230000,754500.0,NWS
1365209,2021-03-09,24.000000,24.459999,23.389999,23.799999,23.799999,632600.0,NWS


In [5]:
# j=1
# k=2
# # the 'midpoints' for each observe and hold period. midpoint in this context is where we switch from the observation period len=j to the holding period len=k
# j_end = pd.date_range(st+pd.DateOffset(months=j*3), et, freq=str(k*3)+"MS")
# j_end.strftime("%Y-%m-%d").to_list()[:10]

In [6]:
# pd.date_range(st+pd.DateOffset(days=j), et)

In [10]:
# def generate_momentum_returns(j=25,k=50):
def generate_daily_timetable(j=10, k=20):
    '''aims to generate the MONTHLY table of times for each observe/hold period. The midpoint in this context is where we switch from the observation period len=j to the holding period len=k'''
    # TODO FINISAH FISODJNF
    j_end = pd.date_range(pd.to_datetime(df.Date.min())+pd.DateOffset(days=j), pd.to_datetime(df.Date.max()), freq=str(k)+"D") # the 'midpoints' for each observe and hold period. 
    j_start = j_end + pd.DateOffset(days=-j) # based on the midpoint, get the start point for each observe and hold period
    k_end = j_end + pd.DateOffset(days=k) # based on the midpoint, get the end point for each observe and hold period
    timetable = pd.DataFrame({'j_start': j_start, 'midpoint': j_end, 'k_end': k_end}).infer_objects()
    return timetable, j, k
timedf, j, k = generate_daily_timetable()
print(timedf.shape[0])
timedf.head() ##NOTE THE K_END VALUE IS NO LONGER EQUAL TO THE NEXT MIDPOINT BUT OFFSET BY 1

204


,j_start,midpoint,k_end
0,2010-01-04,2010-01-14,2010-02-03
1,2010-01-24,2010-02-03,2010-02-23
2,2010-02-13,2010-02-23,2010-03-15
3,2010-03-05,2010-03-15,2010-04-04
4,2010-03-25,2010-04-04,2010-04-24


In [72]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,Name
0,2010-01-04,7.622500,7.660714,7.585000,7.643214,6.583586,493729600.0,AAPL
1,2010-01-05,7.664286,7.699643,7.616071,7.656429,6.594968,601904800.0,AAPL
2,2010-01-06,7.656429,7.686786,7.526786,7.534643,6.490066,552160000.0,AAPL
3,2010-01-07,7.562500,7.571429,7.466071,7.520714,6.478067,477131200.0,AAPL
4,2010-01-08,7.510714,7.571429,7.466429,7.570714,6.521136,447610800.0,AAPL
...,...,...,...,...,...,...,...,...
1365206,2021-03-04,23.020000,23.330000,22.389999,22.770000,22.770000,345900.0,NWS
1365207,2021-03-05,22.940001,23.549999,22.510000,23.500000,23.500000,515500.0,NWS
1365208,2021-03-08,23.500000,23.760000,22.980000,23.230000,23.230000,754500.0,NWS
1365209,2021-03-09,24.000000,24.459999,23.389999,23.799999,23.799999,632600.0,NWS


In [12]:
from itertools import chain
def get_daily_j(id=-1):
    jtimes = pd.DataFrame(list(chain.from_iterable(pd.date_range(timedf["j_start"],timedf["midpoint"]) for _,timedf in timedf.iterrows())), columns=("date",))
    jtimes['date'] = jtimes['date'].astype('datetime64[ns]')
    jtimes['period'] = (jtimes['date'].isin(timedf['midpoint'])).shift(1).cumsum().fillna(0).astype(int)
    df_ = df.copy()
    df_['date'] = pd.to_datetime(df_['Date'])
    df_ = df_.merge(jtimes, how='inner', on='date')
    if id==-1: 
        return df_ # this filters out periods that dont end prettily
    else:
        return df_[df_['period']==id].drop('period', axis=1)
get_j_df = get_daily_j
def get_daily_k(id=-1):
    ktimes = pd.DataFrame(list(chain.from_iterable(pd.date_range(timedf["midpoint"],timedf["k_end"]) for _,timedf in timedf.iterrows())), columns=("date",))
    ktimes['date'] = ktimes['date'].astype('datetime64[ns]')
    ktimes['period'] = (ktimes['date'].isin(timedf['k_end'])).shift(1).cumsum().fillna(0).astype(int)
    df_ = df.copy()
    df_['date'] = pd.to_datetime(df_['Date'])
    df_ = df_.merge(ktimes, how='inner', on='date')
    if id==-1: 
        return df_ # this filters out periods that dont end prettily
    else:
        return df_[df_['period']==id].drop('period', axis=1)
get_daily_k()#.period.plot()

,Date,Open,High,Low,Close,Adj Close,Volume,Name,date,period
0,2010-01-14,7.503929,7.516429,7.465000,7.479643,6.442690,432894000.0,AAPL,2010-01-14,0
1,2010-01-14,30.309999,31.100000,30.260000,30.959999,24.057743,63228100.0,MSFT,2010-01-14,0
2,2010-01-14,129.139999,130.380005,126.400002,127.349998,127.349998,9774900.0,AMZN,2010-01-14,0
3,2010-01-14,292.242249,297.397400,291.696686,295.220215,295.220215,8471720.0,GOOGL,2010-01-14,0
4,2010-01-14,290.859772,295.990570,290.316833,293.823669,293.823669,8511986.0,GOOG,2010-01-14,0
...,...,...,...,...,...,...,...,...,...,...
1430470,2021-03-09,31.990000,32.279999,31.010000,31.059999,31.059999,2735100.0,VNT,2021-03-09,406
1430471,2021-03-09,23.129999,23.139999,22.400000,22.410000,22.410000,4301100.0,UAA,2021-03-09,406
1430472,2021-03-09,26.680000,26.940001,26.320000,26.510000,26.510000,2113200.0,XRX,2021-03-09,406
1430473,2021-03-09,19.240000,19.250000,18.692200,18.709999,18.709999,2110296.0,UA,2021-03-09,406


In [13]:
get_daily_j()#.period.plot()

,Date,Open,High,Low,Close,Adj Close,Volume,Name,date,period
0,2010-01-04,7.622500,7.660714,7.585000,7.643214,6.583586,493729600.0,AAPL,2010-01-04,0
1,2010-01-04,30.620001,31.100000,30.590000,30.950001,24.049969,38409100.0,MSFT,2010-01-04,0
2,2010-01-04,136.250000,136.610001,133.139999,133.899994,133.899994,7599900.0,AMZN,2010-01-04,0
3,2010-01-04,313.788788,315.070068,312.432434,313.688690,313.688690,3908488.0,GOOGL,2010-01-04,0
4,2010-01-04,312.304413,313.579620,310.954468,312.204773,312.204773,3927065.0,GOOG,2010-01-04,0
...,...,...,...,...,...,...,...,...,...,...
750681,2021-02-25,32.490002,33.110001,31.750000,31.910000,31.910000,1604200.0,VNT,2021-02-25,203
750682,2021-02-25,21.940001,22.620001,21.490000,21.590000,21.590000,6187100.0,UAA,2021-02-25,203
750683,2021-02-25,25.309999,25.790001,24.920000,25.350000,25.350000,3828900.0,XRX,2021-02-25,203
750684,2021-02-25,18.280001,18.870001,18.020000,18.100000,18.100000,2355480.0,UA,2021-02-25,203


In [14]:
# def generate_timetable(j=1, k=2):
#     '''aims to generate the MONTHLY table of times for each observe/hold period. The midpoint in this context is where we switch from the observation period len=j to the holding period len=k'''
    
#     j_end = pd.date_range(st+pd.DateOffset(months=j*3), et, freq=str(k*3)+"MS") # the 'midpoints' for each observe and hold period. 
#     j_start = j_end + pd.DateOffset(months=-j*3) # based on the midpoint, get the start point for each observe and hold period
#     k_end = j_end + pd.DateOffset(months=k*3) # based on the midpoint, get the end point for each observe and hold period
#     timetable = pd.DataFrame({'j_start': j_start, 'midpoint': j_end, 'k_end': k_end}).infer_objects()
#     return timetable, j, k
# timedf, j, k = generate_timetable(3, 2)
# timedf.head()

In [15]:
df['Date']

0          2010-01-04
1          2010-01-05
2          2010-01-06
3          2010-01-07
4          2010-01-08
              ...    
1364701    2021-03-03
1364702    2021-03-04
1364703    2021-03-05
1364704    2021-03-08
1364705    2021-03-09
Name: Date, Length: 1364705, dtype: object

In [16]:
midpoint = timedf.iloc[0].midpoint

In [17]:
# def get_j_df(id=-1, j=j):
#     '''this should spit out a dataframe of data in the observational period j given specific id or midpoint value. 
#     if unspecified it throws all of them at you with an extra identifying column "periods"'''
#     if (id==-1): # not really sure why we need this but ill include it. this adds a sector column to the data for future filtering purposes if needed
#         df_i = pd.DataFrame(columns=df.columns.to_list()+['period']) #dummy empty df
#         for i in timedf.index: 
#             # print(timedf.iloc[i].j_start,timedf.iloc[i].midpoint)
#             df_ = df[(timedf.iloc[i].j_start <= pd.to_datetime(df['Date']))&(pd.to_datetime(df['Date']) <= timedf.iloc[i].midpoint)] # gets dates btw start and midpt
#             df_['period'] = i
#             df_i = pd.concat([df_i, df_])

#         return df_i
#     return df[(timedf.iloc[id].j_start <= pd.to_datetime(df['Date']))&(pd.to_datetime(df['Date']) <= timedf.iloc[id].midpoint)] # gets dates btw start and midpt from table
# get_j_df()

In [18]:
# def get_k_df(id=-1, k=k):
#     '''this should spit out a dataframe of data in the holding period k given specific id or midpoint value. 
#     if unspecified it throws all of them at you with an extra identifying column "periods"'''
#     if (id==-1): # we can decide if this is useful later this adds a sector column to the data for future filtering purposes if needed
#         df_i = pd.DataFrame(columns=df.columns.to_list()+['period']) #dummy empty df
#         for i in timedf.index: 
#             # print(timedf.iloc[i].j_start,timedf.iloc[i].midpoint)
#             df_ = df[(timedf.iloc[i].midpoint <= pd.to_datetime(df['Date']))&(pd.to_datetime(df['Date']) <= timedf.iloc[i].k_end)] # gets dates btw midpt and end
#             df_['period'] = i
#             df_i = pd.concat([df_i, df_])

#         return df_i
#     return df[(timedf.iloc[id].midpoint <= pd.to_datetime(df['Date']))&(pd.to_datetime(df['Date']) <= timedf.iloc[id].k_end)] # gets dates btw midpt and end
# get_k_df()

# Data Analysis: using the above functions for building portfolios

In [99]:
timedf,j,k = generate_daily_timetable(j=30, k=60) # with j as 3 and k as 2
timedf

,j_start,midpoint,k_end
0,2010-01-04,2010-02-03,2010-04-04
1,2010-03-05,2010-04-04,2010-06-03
2,2010-05-04,2010-06-03,2010-08-02
3,2010-07-03,2010-08-02,2010-10-01
4,2010-09-01,2010-10-01,2010-11-30
...,...,...,...
63,2020-05-11,2020-06-10,2020-08-09
64,2020-07-10,2020-08-09,2020-10-08
65,2020-09-08,2020-10-08,2020-12-07
66,2020-11-07,2020-12-07,2021-02-05


In [100]:
df0 = get_daily_j(0).set_index('Name') # i dont know why, but you have to set index to name for the groupby's to work
df0['pct_change'] = df0['Adj Close'].groupby('Name').pct_change()
df0['cum_return'] = (df0['pct_change']+1).groupby('Name').cumprod().fillna(1)-1
df0['adj_close_shifted'] = df0['Adj Close'].groupby('Name').shift(3)#.bfill(0) # filing the value with the backfill TODO check this later
df0['adj_change'] = df0['Adj Close']/df0['adj_close_shifted'] #.bfill(0) 

features = df0.groupby('Name').tail(1).sort_values(['cum_return'], ascending=False)
features

,Date,Open,High,Low,Close,Adj Close,Volume,date,pct_change,cum_return,adj_close_shifted,adj_change
Name,,,,,,,,,,,,
ZION,2010-02-03,19.770000,19.780001,18.459999,18.620001,16.271383,10280400.0,2010-02-03,-0.064792,0.396849,16.577236,0.981550
LYV,2010-02-03,11.410000,11.900000,11.410000,11.890000,11.890000,2317700.0,2010-02-03,0.034813,0.303728,11.470000,1.036617
HBAN,2010-02-03,4.880000,4.900000,4.630000,4.710000,3.466631,41303100.0,2010-02-03,-0.044625,0.279891,3.525512,0.983299
DPZ,2010-02-03,11.110000,11.200000,11.030000,11.080000,9.420688,635000.0,2010-02-03,-0.003597,0.276498,9.607740,0.980531
LEN,2010-02-03,16.096361,16.243855,15.644051,15.988201,15.068360,4385101.0,2010-02-03,-0.017523,0.253082,14.234322,1.058594
...,...,...,...,...,...,...,...,...,...,...,...,...
HWM,2010-02-03,30.584707,31.236881,30.202398,30.337332,27.653538,160238.0,2010-02-03,-0.010997,-0.188008,26.038321,1.062032
AMD,2010-02-03,7.830000,7.960000,7.760000,7.810000,7.810000,14521900.0,2010-02-03,-0.012642,-0.194845,7.460000,1.046917
AIG,2010-02-03,23.889999,24.450001,23.709999,23.790001,16.759001,4817500.0,2010-02-03,-0.007509,-0.204082,17.068958,0.981841


In [101]:
# monthly_sp500['PYPL'].head() # TODO we should write about some of the issues of yahoofinance as a datasource, that should be sufficient

In [102]:
# this calculates the winner and loser tickers given n
n = 50
get_percents = lambda n: features.shape[0]//n+1 # this function gets us n percent number of tickers
winner_tickers = features[:get_percents(n)].index
winner_tickers

Index(['ZION', 'LYV', 'HBAN', 'DPZ', 'LEN', 'KEY', 'ILMN', 'DHI', 'AAL',
       'FITB'],
      dtype='object', name='Name')

In [103]:
loser_tickers = features[-get_percents(n):].index
loser_tickers

Index(['IP', 'LRCX', 'MU', 'MPWR', 'URI', 'HWM', 'AMD', 'AIG', 'KLAC', 'TMUS'], dtype='object', name='Name')

In [104]:
# equally weight
weights = np.ones(get_percents(n))/(get_percents(n))

In [105]:
winner_df0 = df0.loc[winner_tickers.to_list()]
winner_df0 = winner_df0.pivot_table(index='Date',columns='Name')['pct_change'].fillna(0) # 
winner_df0
# winner_df0.dot(weights) # gives us the percent change of the portfolios
winner_performance = (winner_df0.dot(weights)+1).cumprod() - 1 # gets cumulative return for a period
winner_performance

Date
2010-01-04    0.000000
2010-01-05    0.034084
2010-01-06    0.059467
2010-01-07    0.119671
2010-01-08    0.119166
2010-01-11    0.124764
2010-01-12    0.135642
2010-01-13    0.167968
2010-01-14    0.192803
2010-01-15    0.175896
2010-01-19    0.186416
2010-01-20    0.185581
2010-01-21    0.191856
2010-01-22    0.175227
2010-01-25    0.199872
2010-01-26    0.202174
2010-01-27    0.223839
2010-01-28    0.235507
2010-01-29    0.235434
2010-02-01    0.258644
2010-02-02    0.293070
2010-02-03    0.267778
dtype: float64

In [106]:
def get_cum_return(data):
    '''gets cumulative return based on adjusted closing price of all tickers in input dataframe'''
    df_ = data.set_index('Name')

    # df_['pct_change'] = df_['Adj Close'].groupby('Name').pct_change()
    df_['pct_change'] = df_['Adj Close'].groupby('Name').pct_change()
    df_['cum_return'] = (df_['pct_change']+1).groupby('Name').cumprod().fillna(1)-1
    df_['adj_close_shifted'] = df_['Adj Close'].groupby('Name').shift(3)#.bfill(0) # filing the value with the backfill TODO check this later
    df_['adj_change'] = df_['Adj Close']/df_['adj_close_shifted'] #.bfill(0) 
    df_['adj_close_shifted'] = df_['Adj Close'].groupby('Name').shift(3)#.bfill(0) # filing the value with the backfill TODO check this later
    df_['adj_change'] = df_['Adj Close']/df_['adj_close_shifted'] #.bfill(0) 
    
    return df_

def get_percents(n): 
    '''this function gets us n percent number of tickers'''
    return features.shape[0]//n+1

def get_portfolios(period=0, n=50):   
    '''returns the portfolios we want from a particular period'''
    ## Gets the portfolio(s) in question we want to look at
    data = get_cum_return(get_daily_j(period))
    features = data.groupby('Name').tail(1).sort_values(['cum_return'], ascending=False) # sorts tickers by cumulative return

    winner_tickers = features[:get_percents(n)].index # we get the top/bottom n percent tickers
    loser_tickers = features[-get_percents(n):].index # only winner and loser portfolios for now, we could expand later
    # TODO add more portfolios if we have time
    # print(len(winner_tickers), len(loser_tickers))
    print('.', end='')
    return (winner_tickers.to_list(), loser_tickers.to_list())

def get_portfolio_performance(period=0, n=2, weights=None, hold=False):
    '''Gets all portfolios' cumulative return performance based on n period 
    keyword args:
    period      -- the nth period of data we are looking at, default 0
    portfolio   -- the particular type of portfolio we want to be looking at (winner or loser, etc) default winner/momentum
    n           -- the percentage of tickers we want to be looking at
    weights     -- how to weight the portfolio values. if unspecified (None) we assume equal weighting in the portfolio
    hold        -- if true returns the performance evaluation of the holding period k, else returns the performance of the observation period j
    '''
    winner_tickers, loser_tickers = get_portfolios(period, n)

    ## evaluates the performance of portfolios on either hold or observational data
    eval_df = get_cum_return(get_k_df(period)) if hold else get_cum_return(get_daily_j(period))# we get the return from the hold period
    
    weights = np.ones(get_percents(n))/(get_percents(n)) if not weights else weights # set weights
    print('weights', len(weights))
    winner_eval = eval_df.loc[winner_tickers] # first the winners
    winner_eval = winner_eval.pivot_table(index='Date',columns='Name')['pct_change'].fillna(0)
    winner_performance = (winner_eval.dot(weights)+1).cumprod() - 1 # cumulative return

    loser_eval = eval_df.loc[loser_tickers] # then the losers
    loser_eval = loser_eval.pivot_table(index='Date',columns='Name')['pct_change'].fillna(0)
    loser_performance = (loser_eval.dot(weights)+1).cumprod() - 1 # cumulative return

    output = pd.DataFrame({'winners': winner_performance, 'losers': loser_performance})
    return output
get_portfolio_performance()

.weights 226


,winners,losers
Date,,
2010-01-04,0.000000,0.000000
2010-01-05,0.007279,0.000804
2010-01-06,0.013506,-0.001028
2010-01-07,0.025162,-0.000676
2010-01-08,0.027712,0.002783
2010-01-11,0.032591,0.003581
2010-01-12,0.027366,-0.012536
2010-01-13,0.038880,-0.003022
2010-01-14,0.043425,-0.004658


In [107]:
get_portfolios(1)

.

(['CNC', 'PVH', 'INCY', 'AIG', 'PXD', 'C', 'LVS', 'PFG', 'HST', 'CBRE'],
 ['MOS', 'DVN', 'NRG', 'ABMD', 'BSX', 'IT', 'COG', 'STX', 'LDOS', 'CF'])

In [108]:
# 10/500
# pf = pd.DataFrame({'idx':timedf.index})
# pf['mom'], pf['rev'] = zip(*pf['idx'].map(get_portfolios))
# pf

In [109]:
portfolios = {'mom':{}, 'rev':{}}
for i in timedf.index:
    # print(i)
    mom, rev = get_portfolios(i)
    portfolios['mom'][i] = mom
    portfolios['rev'][i] = rev
# portfolios['mom'][0]
pf = timedf.merge(pd.DataFrame(portfolios), left_index=True, right_index=True)
pf.head()

....................................................................

,j_start,midpoint,k_end,mom,rev
0,2010-01-04,2010-02-03,2010-04-04,"[ZION, LYV, HBAN, DPZ, LEN, KEY, ILMN, DHI, AA...","[IP, LRCX, MU, MPWR, URI, HWM, AMD, AIG, KLAC,..."
1,2010-03-05,2010-04-04,2010-06-03,"[CNC, PVH, INCY, AIG, PXD, C, LVS, PFG, HST, C...","[MOS, DVN, NRG, ABMD, BSX, IT, COG, STX, LDOS,..."
2,2010-05-04,2010-06-03,2010-08-02,"[AAL, DAL, UHS, ALK, NTAP, TMUS, AAP, NFLX, UA...","[LEN, MCO, DHI, LYB, MA, LYV, BR, PHM, HAL, BKNG]"
3,2010-07-03,2010-08-02,2010-10-01,"[URI, WY, CBRE, AAL, TXT, TSLA, IPG, RCL, UAL, F]","[HSY, NLOK, MCK, CBOE, AKAM, TMO, BSX, MU, WDC..."
4,2010-09-01,2010-10-01,2010-11-30,"[KMX, ULTA, URI, BBY, PVH, UAA, INCY, AMD, WBA...","[JCI, AAL, LNC, CPRT, CRM, MPWR, ETR, CBOE, MT..."


In [110]:
def get_portfolio_pct(lst, date, weights=None):
    '''given list of tickers, generates a EQUAL WEIGHTED INDEX portfolio and aggregates their performance'''
    portfolio_df = df[df['Name'].isin(lst)] #filters df by lst
    portfolio_df = get_cum_return(portfolio_df)[['Date','pct_change']] # we get cum return
    portfolio_df = portfolio_df.groupby('Date').mean()
    value = portfolio_df[pd.to_datetime(portfolio_df.index)==pd.to_datetime(date)].values
    # print('.', end='')
    return 0 if (value.shape[0]==0 or np.isnan(value[0,0])) else value[0,0] # will need to spend time staring at the output to see if most columns are good
    # im thinking the only good value we can get out of this is percent change and maybe cum_return, ohlc columns are garbage
# temp = get_portfolio_pct(['NFLX', 'LVS', 'URI', 'CMG', 'FFIV'], dpf.Date[0])
# temp


SyntaxError: invalid syntax (<ipython-input-110-eea725d9058c>, line 6)

In [ ]:
dpf = pd.DataFrame({'Date':pd.date_range(df.Date.min(), df.Date.max(), freq="D")}).merge(pf[['k_end', 'mom', 'rev']].reset_index(), how='left', left_on='Date', right_on='k_end').bfill().ffill()
dpf['mom_pct_change'] = dpf.apply(lambda x: get_portfolio_pct(x.mom, x.Date), axis=1)
dpf['rev_pct_change'] = dpf.apply(lambda x: get_portfolio_pct(x.rev, x.Date), axis=1)
dpf['index'] = dpf['index'].astype(int)
dpf = dpf.drop(['mom', 'rev', 'k_end'], axis=1)
dpf['mom_return'] = ((dpf['mom_pct_change']+1).cumprod()-1) * 100
dpf['rev_return'] = ((dpf['rev_pct_change']+1).cumprod()-1) * 100
dpf[['mom_return', 'rev_return', 'Date']].set_index('Date').plot(figsize=(12,8))
dpf

In [ ]:
dpf

In [ ]:
spc = pdr.get_data_yahoo("^GSPC", start="2009-10-01")
spc

In [ ]:
import matplotlib.ticker as mtick

fig = plt.figure(1, (12,8))
ax = fig.add_subplot(1,1,1)
fmt = '%.0f%%' # Format you want the ticks, e.g. '40%'
xticks = mtick.FormatStrFormatter(fmt)
ax.yaxis.set_major_formatter(xticks)
spc['pctc'] = spc['Adj Close'].pct_change()
spc['cumprod'] =((spc['pctc']+1).cumprod()-1)*100
sp500 = spc.merge(dpf, how='left', left_index=True, right_on='Date')
sp500[['Date', 'mom_return', 'rev_return','cumprod']].set_index('Date').plot.area(ax=ax,stacked=False,figsize=(12,8)) # plotted
ax.legend(["Momentum", "Reversal", "S&P500"])
# fig.savefig('performance.png')

In [ ]:
sp500 = sp500.drop(['mom_return', 'rev_return'], axis=1) # mom_retunr and rev_return, beware of use
sp500 = sp500.reset_index(drop=True).set_index('Date')
sp500

In [ ]:

sp500.ta.cci(length=4,append=True)
sp500.ta.kdj(length=4,append=True)
sp500.ta.rsi(length=4,append=True)
sp500.ta.bop(length=4,append=True)
sp500.ta.willr(length=4,append=True)
sp500.ta.pdist(length=4,append=True)
sp500.ta.kc(length=4,append=True)
sp500.ta.adx(length=4,append=True)
sp500.ta.qstick(length=4,append=True)
sp500.ta.roc(length=4,append=True)
# sp500.ta.ao(length=4,append=True)
# sp500.ta.macd(fast=4,slow=8,append=True)
sp500.ta.stdev(length=4,append=True)
sp500.ta.pvol(append=True)
sp500.ta.efi(length=4, append=True)
sp500

In [ ]:
sp500.columns

In [ ]:
sp500[['CCI_4_0.015',
       'K_4_3', 'D_4_3', 'J_4_3', 'RSI_4', 'BOP', 'WILLR_4', 'PDIST',
       'KCLe_4_2', 'KCBe_4_2', 'KCUe_4_2', 'ADX_4', 'DMP_4', 'DMN_4', 'QS_4',
       'ROC_4', 'STDEV_4', 'PVOL', 'EFI_4']][:20]

In [ ]:
# pd.date_range(df.Date.min(), df.Date.max())
sp500 = sp500[sp500.index.isin(pd.date_range(timedf.j_start.min(), timedf.k_end.max()))]
sp500

In [ ]:
timedf

In [98]:
sp500.to_csv('daily_j20_k40.csv')

In [51]:
sp500 = sp500.fillna(0)
# sp500['date'] = sp500.index
sp500['Date'] = sp500.index
df = sp500
get_daily_j()

ValueError: 'date' is both an index level and a column label, which is ambiguous.

In [45]:
per0 = get_j_df(0)
per0.head()

NameError: name 'get_j_df' is not defined

In [89]:
per0['mom_cum_ret'] = (per0['mom_pct_change']+1).cumprod()-1
per0['rev_cum_ret'] = (per0['rev_pct_change']+1).cumprod()-1
per0

KeyError: 'mom_pct_change'

leptokurtic distribution
the only things we can do is buy, hold, or leave for rev or momentum
<br><br>choices:
- momentum
- reversal
- neither

In [60]:
spc

,Open,High,Low,Close,Adj Close,Volume,pctc,cumprod
Date,,,,,,,,
2010-01-01,1116.560059,1150.449951,1071.589966,1073.869995,1073.869995,90947580000,NaN,NaN
2010-02-01,1073.890015,1112.420044,1044.500000,1104.489990,1104.489990,84561340000,0.028514,2.851369
2010-03-01,1105.359985,1180.689941,1105.359985,1169.430054,1169.430054,103683550000,0.058796,8.898662
2010-04-01,1171.229980,1219.800049,1170.689941,1186.689941,1186.689941,116741910000,0.014759,10.505922
2010-05-01,1188.579956,1205.130005,1040.780029,1089.410034,1089.410034,127662780000,-0.081976,1.447106
...,...,...,...,...,...,...,...,...
2020-12-01,3645.870117,3760.199951,3633.399902,3756.070068,3756.070068,96056410000,0.037121,249.769533
2021-01-01,3764.610107,3870.899902,3662.709961,3714.239990,3714.239990,105548790000,-0.011137,245.874268
2021-02-01,3731.169922,3950.429932,3725.620117,3811.149902,3811.149902,98596960000,0.026091,254.898630


In [61]:
timedf[0]

,j_start,midpoint,k_end
0,2010-02-01,2010-11-01,2011-05-01
1,2010-08-01,2011-05-01,2011-11-01
2,2011-02-01,2011-11-01,2012-05-01
3,2011-08-01,2012-05-01,2012-11-01
4,2012-02-01,2012-11-01,2013-05-01
5,2012-08-01,2013-05-01,2013-11-01
6,2013-02-01,2013-11-01,2014-05-01
7,2013-08-01,2014-05-01,2014-11-01
8,2014-02-01,2014-11-01,2015-05-01
9,2014-08-01,2015-05-01,2015-11-01
